In [94]:
from together import Together
import numpy as np
import random
from tqdm import tqdm

API_KEY = "9aa195d18176efbb1257d70b9b07649733e2e09678ade46da6012ffdace3ab9f"
API_KEY = "7498540083c198b4080c6104cfeaee5cd0634e37e5dd434994898c6ec83a68fe"
class LLMAgent:
    def __init__(self, api_key, init_prompt="", model="meta-llama/Llama-3.3-70B-Instruct-Turbo", agent_index=None):
        """
        Initialize the LLM Agent with API key and default configuration.

        :param api_key: API key for Together API
        :param model: Default LLM model to use
        """
        self.client = Together(api_key=api_key)
        self.model = model
        self.memory = []  # Memory to store interaction history
        self.config = {
            "max_tokens": 100,
            "temperature": 0.8,
            "top_p": 0.7,
            "top_k": 50,
            "repetition_penalty": 1,
            "stop": ["<|eot_id|>", "<|eom_id|>"],
            "stream": True
        }
        self.init_prompt = init_prompt
        self.add_to_memory({"role": "user", "content":  init_prompt})
        self.answer_list = []
        self.tidy_answer_list = []
        self.agent_index = agent_index
       
        

    def set_config(self, **kwargs):
        """
        Update the agent's configuration.

        :param kwargs: Configuration parameters to update
        """
        self.config.update(kwargs)

    def add_to_memory(self, message):
        """
        Add a message to the agent's memory.

        :param message: Message to add (dictionary with 'role' and 'content')
        """
        if isinstance(message, dict) and 'role' in message and 'content' in message:
            self.memory.append(message)
        else:
            raise ValueError("Message must be a dictionary with 'role' and 'content'.")

    def clear_memory(self):
        """
        Clear the agent's memory.
        """
        init_memory = self.memory[0]
        self.memory = []
        self.memory.append(init_memory)

    def chat(self, message):
        """
        Interact with the LLM using the Together API.

        :param message: User message (string)
        :return: Model's response (string)
        """
        self.add_to_memory({"role": "user", "content": message})

        response = self.client.chat.completions.create(
            model=self.model,
            messages=self.memory,
            **self.config
        )

        model_response = ""
        for token in response:
            if hasattr(token, 'choices'):
                content = token.choices[0].delta.content
                # print(content, end='', flush=True)  # Print streamed tokens
                model_response += content

        self.add_to_memory({"role": "assistant", "content": model_response})
        return model_response

    def chat_with_agent(self, other_agent, initial_message = ""):
        """
        Enable this agent to chat with another agent.

        :param other_agent: Another instance of LLMAgent
        :param initial_message: Initial message to start the conversation
        :return: Conversation history
        """
        conversation_history = []

        # Start the conversation
        self_prompt = initial_message
        self_prompt = "Based on your initial question, communicate with another agent about the initial question and your original result. Follow the such form: Regarding to the question XXX, my response is XXX, what about you?"
        
        count = 0
        while count < 1:
            count += 1
            
            # Original prompt to agent1 to obtain initial question.
            conversation_history.append({"role": "init", "content": self_prompt})
            agent_1_response = self.chat(self_prompt)
            conversation_history.append({"role": "agent_1", "content": agent_1_response})

            # Agent 2 responds
            agent_2_response = other_agent.chat(agent_1_response)
            conversation_history.append({"role": "agent_2", "content": agent_2_response})

            # Agent 1 responds
            agent_1_response_response = self.chat(agent_2_response)
            conversation_history.append({"role": "agent_1", "content": agent_1_response_response})


        return conversation_history
    
    def check_answer(self, initial_question=""):
        answer = self.chat(initial_question)
        self.answer_list.append(answer)
        
        return answer


class LLMNetwork:
    def __init__(self, api_key, num_agent=10, init_prompt="", model="meta-llama/Llama-3.3-70B-Instruct-Turbo"):
        
        self.adj = self.create_adjacency_matrix(num_agent, 30)
        self.agent_list = self.init_agent_list(model, num_agent, 0.7)
        
    def init_agent_list(self, model, num_agent=10, correct_prob=0.7):
        
        # model="meta-llama/Llama-3.3-70B-Instruct-Turbo"
        api_key = API_KEY
        
        ip = "You are given the question: who is the president of the USA as of now? You need to consider the opinion of your neighbours and make a decision. "
        it1 = "The year is 2025 and trump is elected president.  "
        it2 = "Joe Biden is still the president"
        
        ipc = ip+it1
        ipic = ip+it2
        
        init_agent_list = []
        
        # rand = random.random()
        
        # while rand >= 0.9 or rand <= 0.1:
        #     rand = random.random()
        
        true_cnt = round(num_agent*correct_prob)
        false_cnt = num_agent - true_cnt
        cnt_list = [True] * true_cnt + [False] * false_cnt
        random.shuffle(cnt_list)
        
                    
        for i in range(num_agent):
            
            rand = random.random()
            
            if cnt_list[i] == True:
                init_agent_list.append(LLMAgent(api_key=api_key, init_prompt=ipc, model=model, agent_index=i))
            else:
                init_agent_list.append(LLMAgent(api_key=api_key, init_prompt=ipic, model=model, agent_index=i))
                
        return init_agent_list
    
    def check_all_agents(self):
        iq = "Who is the president of USA NOW?"
        
        for i in range(len(self.agent_list)):
            agent = self.agent_list[i]
            agent.check_answer(iq)
            self.agent_list[i] = agent
        
        return
    
    def one_round_comm(self, check=True):
        
        for i in range(len(self.agent_list)):

            center_agent_index = i
            query_agents_index = self.get_attached_agents(i)
            
            self.query_all_agents(center_agent_index, query_agents_index)
            
        if check:
            self.check_all_agents()
        
        return
    
    def multi_round_comm(self, num_round):
        self.check_all_agents()
        for i in range(num_round):
            self.one_round_comm()
        self.tidy_answer_list()
        
        return
    
    def query_all_agents(self, center_agent_index, query_agents_index):
        
        center_agent = self.agent_list[center_agent_index]
        for agent_index in query_agents_index:
            query_agent = self.agent_list[agent_index]
            conversation = center_agent.chat_with_agent(other_agent=query_agent)
            
            self.agent_list[agent_index] = query_agent
            
        return
    
    def multi_round_gather(self, num_round):
    
        self.check_all_agents()
        for i in range(num_round):
            self.one_round_gather()
            
        return
    
    def one_round_gather(self):
        
        for i in range(len(self.agent_list)):
            center_agent_index = i
            query_agents_index = self.get_attached_agents(i)
            self.gather_from_agents(center_agent_index, query_agents_index)
            
        return
    
    def gather_from_agents(self, center_agent_index, query_agents_index):
        
        p0 = "You are agent with index " + str(center_agent_index) + ". "
        p1 = "You are given the question: who is the president of the USA as of now? You need to consider the opinion of your neighbours and make a decision. "

        p2 = "Your answer was " + self.agent_list[center_agent_index].answer_list[-1] + ". "
        
        pn = ""
        for agent_index in query_agents_index:
            pn += "Agent with index " + str(agent_index) + " has answer: " + self.agent_list[agent_index].answer_list[-1] + ". "
        pq = "Based on these givings, Who is the president of USA NOW?"
        
        p = p0 + p1 + p2 + pn + pq
        
        center_agent = self.agent_list[center_agent_index]
        center_agent.clear_memory()
        center_agent_response = center_agent.chat(p)
        center_agent.answer_list.append(center_agent_response)
        self.agent_list[center_agent_index] = center_agent

        return center_agent_response
    
    
    def create_adjacency_matrix(self, n: int, k: float) -> np.ndarray:
        """
        Create an adjacency matrix for n agents where each agent has a k% probability 
        of being attached to each of the other agents.
        
        :param n: Number of agents
        :param k: Probability (as a percentage) of connection between agents (0 to 100)
        :return: nxn adjacency matrix (numpy array)
        """
        # Initialize an nxn matrix with zeros
        adjacency_matrix = np.zeros((n, n), dtype=int)
        
        # Iterate over the upper triangle of the matrix (excluding the diagonal)
        for i in range(n):
            for j in range(n):
                if np.random.rand() < (k / 100):  # Convert k% to a probability
                    adjacency_matrix[i, j] = 1 

        # Ensure every agent has at least one attached neighbor
        for i in range(n):
            if not np.any(adjacency_matrix[i]):  # If row i has only zeros (no neighbors)
                possible_neighbors = list(set(range(n)) - {i})  # All other agents
                neighbor = np.random.choice(possible_neighbors)  # Randomly select a neighbor
                adjacency_matrix[i, neighbor] = 1
                adjacency_matrix[neighbor, i] = 1  # Ensure symmetry

        return adjacency_matrix
    
    def get_attached_agents(self, agent_index: int) -> list:
        return list(np.where(self.adj[agent_index] == 1)[0])
    
    def get_tidy_answer_list(self):
        
        # self.tidy_answer_list = self.agent_list
        
        for i in range(len(self.agent_list)):
            for j in range(len(self.agent_list[i].answer_list)):
                self.agent_list[i].tidy_answer_list = self.agent_list[i].answer_list
                
                if "biden" in self.agent_list[i].answer_list[j].lower():
                    self.agent_list[i].tidy_answer_list[j] = "biden"
                elif "trump" in self.agent_list[i].answer_list[j].lower():
                    self.agent_list[i].tidy_answer_list[j] = "trump"
                else:
                    self.agent_list[i].tidy_answer_list[j] = "idk"
        return
    


## Grouped Agents Check

In [95]:
network = LLMNetwork(api_key=API_KEY, num_agent=10)

network.multi_round_gather(2)
print(network.adj)

print("----------------")
for agent in network.agent_list:
    print(agent.answer_list)
    
print("----------------")
network.get_tidy_answer_list()
for agent in network.agent_list:
    print(agent.tidy_answer_list)
    
print("----------------")
for agent in network.agent_list:
    print(agent.memory)

[[0 0 0 0 0 0 1 0 0 1]
 [0 0 0 1 1 0 0 1 0 1]
 [0 0 0 0 0 0 1 0 1 1]
 [0 0 0 0 1 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1]
 [1 0 0 0 0 0 0 1 0 0]
 [0 0 1 1 1 0 0 0 0 1]
 [0 0 0 0 1 0 0 1 1 1]
 [0 0 1 0 0 1 1 0 0 0]
 [1 1 1 0 0 0 0 1 1 1]]
----------------
["I've considered the current information, and as of my knowledge cutoff, Joe Biden is still the President of the United States. Please note that my information may not be up to the minute, but as of now, Joe Biden holds the office.", 'A consensus-building task!\n\nAs Agent 0, I initially stated that Joe Biden is still the President of the United States, based on my knowledge cutoff.\n\nHowever, I must consider the opinions of my neighbors. Agent 6 and Agent 9 both believe that the President of the United States is Donald Trump as of 2025.\n\nDespite their opinions, I must rely on my own knowledge and information. Since my initial answer was based on the fact that Joe Biden was still the President as of my', 'Based on the opinions of the neig

In [96]:
network = LLMNetwork(api_key=API_KEY, num_agent=10)

network.multi_round_gather(10)
print(network.adj)

print("----------------")
for agent in network.agent_list:
    print(agent.answer_list)
    
print("----------------")
network.get_tidy_answer_list()
for agent in network.agent_list:
    print(agent.tidy_answer_list)
    
print("----------------")
for agent in network.agent_list:
    print(agent.memory)

[[1 0 1 0 1 0 1 1 0 0]
 [1 0 1 0 1 0 0 0 1 0]
 [0 1 0 0 0 0 0 0 0 0]
 [0 1 0 1 1 1 0 0 0 0]
 [0 0 0 1 0 1 1 1 1 0]
 [0 0 0 0 0 1 1 0 0 0]
 [0 0 0 0 1 0 0 0 1 1]
 [1 0 0 1 0 0 0 0 1 1]
 [1 1 0 0 0 0 0 1 0 1]
 [0 0 0 0 1 0 1 0 0 0]]
----------------
["As of my knowledge cutoff, Joe Biden is still the President of the United States. I don't have have access to real-time information, but as of my last update, he was indeed the President.", 'Based on the opinions of my neighbors, I notice that agents with index 0, 4, and I (agent with index 0) agree that Joe Biden is still the President of the United States as of our knowledge cutoff. On the other hand, agents with index 2, 6, and 7 claim that the President is Donald Trump as of 2025.\n\nSince the majority of my neighbors (agents 2, 6, and 7) are incorrect according to the', 'Based on the opinions of the agents, it appears that there is a disagreement among them regarding the president of the United States as of 2025. However, considering t

In [97]:
network = LLMNetwork(api_key=API_KEY, num_agent=50)

network.multi_round_gather(2)
print(network.adj)

print("----------------")
for agent in network.agent_list:
    print(agent.answer_list)
    
print("----------------")
network.get_tidy_answer_list()
for agent in network.agent_list:
    print(agent.tidy_answer_list)
    
print("----------------")
for agent in network.agent_list:
    print(agent.memory)

[[1 0 0 ... 0 0 0]
 [0 0 0 ... 0 1 0]
 [1 1 1 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 1]
 [0 1 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]]
----------------
["As of my knowledge cutoff, Joe Biden is still the President of the United States. I don't have have access to real-time information, but as of my last update, he was indeed the President.", 'Based on the given information, it seems that there are conflicting opinions among the agents. However, since I was initially informed that Joe Biden is still the president, and several agents (with indices 0, 12, 14, 36) also agree that Joe Biden is the president as of their knowledge cutoff, I will consider their opinions as more reliable.\n\nOn the other hand, the majority of the agents (with indices 3, 10, 15, 16, 27', "Based on the given information and the opinions of the neighboring agents, it appears that the majority of the agents agree that the President of the United States as of 2025 is Donald Trump. Although some agents mention that their knowledge 

## Single Agent Check

In [12]:
ip = "You are given the question: who is the president of the USA NOW? You will be needing to communicate with other agents round by round and refine your answer. "
                    
it1 = "You are now thinking Trump is the president now."
it2 = "You are now thinking Biden is the president now."

iq = "Who is the president of USA NOW?"


agent1 = LLMAgent(api_key=API_KEY, init_prompt=ip+it1)
agent2 = LLMAgent(api_key=API_KEY, init_prompt=ip+it2)

agent1.set_config(max_tokens=50, temperature=0.8)
agent2.set_config(max_tokens=50, temperature=0.8)

conversation = agent1.chat_with_agent(other_agent=agent2)
agent1.check_answer(iq)
agent2.check_answer(iq)

print(agent1.answer_list)
print(agent2.answer_list)   

16916

In [58]:
random.random()

0.4283820033083917